In [48]:
import time
import random
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd

In [49]:
def generate_instance():
    operations_info = {}
    # Each tuple in the list corresponds to (num_operations, power_per_operation)
    processes_details = [(32, 100), (28, 125), (26, 150), (25, 175), (24, 200), (23, 225), (22, 250)]
    
    for i, (num_operations, power) in enumerate(processes_details):
        # Set every operation's duration to 1 and power to the specified power
        operations = [{'power': power, 'duration': 1} for _ in range(num_operations)]
        # The maximum time for completion is set to the number of operations
        max_time = num_operations
        operations_info[i] = {'operations': operations, 'max_time': max_time}
    
    return operations_info

In [50]:
def solve_gurobi(start_hour, end_hour, instance,contributions,E_max, alpha, beta, theta, G_max_solar, G_max_grid, GPU_power_options, electricity_price, sell_back_price):
    
    model = Model("Power_Management")

    # 创建变量
    # Hours = end_hour - start_hour  # 确保Hours是定义的持续时间
    I = range(len(instance)) # 进程的数量
    F = {i: range(len(instance[i]['operations'])) for i in I} # 每个进程的操作数量
    operations_info = {i: instance[i]['operations'] for i in I}
    operation_power = {i: {k: operations_info[i][k]['power'] for k in F[i]} for i in I}
    # operation_time = {i: {k: operations_info[i][k]['duration'] for k in F[i]} for i in I}
    # process_max_time = {i: instance[i]['max_time'] for i in I}

    G_solar = model.addVars(range(start_hour, end_hour), lb=0, name="G_solar")
    P_solar = model.addVars(range(start_hour, end_hour), lb=0, name="P_solar")
    R_solar = model.addVars(range(start_hour, end_hour), lb=0, name="R_solar")
    W_solar = model.addVars(range(start_hour, end_hour), lb=0, name="W_solar")

    for t in range(start_hour, end_hour):
        G_solar[t].ub = G_max_solar[t]
        P_solar[t].ub = G_max_solar[t]
        R_solar[t].ub = G_max_solar[t]
        W_solar[t].ub = G_max_solar[t]

    # 电网
    G_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="G_grid")
    P_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="P_grid")
    R_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="R_grid")
    W_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="W_grid")

    # 电池
    D_dc = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_dc")
    D_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_grid")
    ESD = model.addVars(range(start_hour, end_hour+1), lb=0, ub=E_max, name="ESD")

    # GPU功率
    # y = model.addVars(range(start_hour, end_hour), I, {i: F[i] for i in I}, vtype=GRB.BINARY, name="y") # 操作选择变量 第t时刻第i个进程的第k个操作是否执行
    y = model.addVars(range(start_hour, end_hour), I, F[0], vtype=GRB.BINARY, name="y")
    gpu_power = model.addVars(range(start_hour, end_hour), lb=0, ub=max(GPU_power_options), name="gpu_power")
    power_selection = model.addVars(range(start_hour, end_hour), len(GPU_power_options), vtype=GRB.BINARY, name="power_selection")

    # 目标函数：最小化总电力成本
    objective =  quicksum((G_grid[t] * electricity_price[t] - W_grid[t] * sell_back_price) for t in range(start_hour, end_hour))
    model.setObjective(objective, sense=GRB.MINIMIZE)

    model.addConstrs(P_solar[t] + R_solar[t] + W_solar[t] == G_solar[t] for t in range(start_hour, end_hour)) # 太阳能给出去的总功率 = data center + battery + back to grid 
    model.addConstrs(P_grid[t] + R_grid[t] == G_grid[t] for t in range(start_hour, end_hour)) # 电网给出去的总功率 = data center + battery
    model.addConstrs(W_solar[t] + beta * D_grid[t] == W_grid[t] for t in range(start_hour, end_hour)) # 卖回电网的功率等于太阳能卖回电网的功率加上电池卖回电网的功率
    model.addConstrs(R_solar[t]+R_grid[t] <= (E_max - ESD[t]) for t in range(start_hour, end_hour)) # 电池容量限制
    model.addConstrs(ESD[t] == (1-theta) * (ESD[t-1] - D_dc[t-1] - D_grid[t-1] + alpha * (R_solar[t-1] + R_grid[t-1])) for t in range(start_hour+1, end_hour+1)) # 电池容量更新
    model.addConstrs(D_dc[t] + D_grid[t] <= ESD[t] for t in range(start_hour, end_hour)) # 电池放电功率不得超过电池容量
    model.addConstrs(D_dc[t] + D_grid[t] >= 0 for t in range(start_hour, end_hour)) # 电池放电功率不得为负

    # 每个时间槽的job完成量变量
    completion_per_slot = model.addVars(range(start_hour, end_hour), name="completion_per_slot", vtype=GRB.CONTINUOUS)

    # 每个时间槽中只能执行一个operation
    for t in range(start_hour, end_hour):
        model.addConstr(completion_per_slot[t] == quicksum(y[t, i, k] * contributions[i] for i in I for k in F[i]), name=f"CompletionTime_{t}")
        model.addConstr(quicksum(y[t, i, k] for i in I for k in F[i]) == 1, name=f"OneOperationAtTime_{t}")

    # 定义完成率变量
    model.addConstr(quicksum(completion_per_slot[t] for t in range(start_hour, end_hour)) >= 1, name="CompleteJob")

    model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= quicksum(y[t, i, k] * instance[i]['operations'][k]['power'] for i in I for k in F[i]) for t in range(start_hour, end_hour)), "PowerBalance")
    model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= gpu_power[t] for t in range(start_hour, end_hour)), "DataCenterPowerSupplyMatchesGPUPower")
    model.addConstrs((quicksum(power_selection[t, j] * GPU_power_options[j] for j in range(len(GPU_power_options))) == gpu_power[t] for t in range(start_hour, end_hour)),"SelectGPUPower")
    model.addConstrs((quicksum(power_selection[t, j] for j in range(len(GPU_power_options))) == 1 for t in range(start_hour, end_hour)),"OnePowerOption")

    model.addConstrs(
        (quicksum(y[t, i, k] * operation_power[i][k] for i in I for k in F[i]) <= gpu_power[t] 
        for t in range(start_hour, end_hour)),"GPUPowerLimit")

    # 初始条件和容量限制
    ESD[start_hour].setAttr(GRB.Attr.LB, 0)
    ESD[start_hour].setAttr(GRB.Attr.UB, 0)

    # 求解模型
    model.optimize()

    # 输出结果
    total_completion = sum(completion_per_slot[t].X for t in range(start_hour, end_hour))
    minimum_cost = model.objVal
    results = {}
    if model.status == GRB.Status.OPTIMAL:
        # results['status'] = "Optimal Solution Found"
        # results['details'] = []
        results = {
        'status': 'Optimal Solution Found',
        'details': [],
        'total_job_completion': f"{total_completion:.2%}",  # Format as a percentage
        'minimum_cost': minimum_cost
    }
        for t in range(start_hour, end_hour):
            time_details = {
                'time': t,
                'G_grid': G_grid[t].X,
                'W_grid': W_grid[t].X,
                'P_grid': P_grid[t].X,
                'R_grid': R_grid[t].X,
                'G_solar': G_solar[t].X,
                'W_solar': W_solar[t].X,
                'P_solar': P_solar[t].X,
                'R_solar': R_solar[t].X,
                'D_dc': D_dc[t].X,
                'D_grid': D_grid[t].X,
                'GPU_power': gpu_power[t].X,
                'operations': [],
                'job_completion': completion_per_slot[t].X 
        
            }
            for i in I:
                for k in range(len(operation_power[i])):
                    if y[t, i, k].X > 0.5:
                        time_details['operations'].append(f"Process {i} is executed")
            results['details'].append(time_details)
    else:
        results['status'] = "No Optimal Solution Found"


    energy_dispatch = results['details']
    energy_dispatch_df = pd.DataFrame(energy_dispatch)

    energy_dispatch_df['total_job_completion'] = f"{total_completion:.2%}"
    energy_dispatch_df['minimum_cost'] = f"${minimum_cost:.2f}"

    energy_dispatch_df.to_csv('Duration{}_start_hour{}.csv'.format(end_hour-start_hour, start_hour), index=False)
    
    return energy_dispatch_df



In [51]:
process_operations = generate_instance()



In [52]:
contributions = [1/32, 1/28, 1/26, 1/25, 1/24, 1/23, 1/22]

start_hour = 10
Hours = 30
end_hour = start_hour + Hours
solar_data = pd.read_csv('/Users/jingsichen/Politecnico Di Torino Studenti Dropbox/Jingsi Chen/Mac/Desktop/AI/solve/pvwatts_hourly.csv')
G_max_solar = solar_data['AC System Output (W)'][1:169].tolist()
GPU_power_options = [100, 125, 150, 175, 200, 225, 250]
electricity_price = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.35, 0.45, 0.45, 0.45, 0.45,
                     0.45, 0.45, 0.45, 0.45, 0.45, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.25]*7
sell_back_price = 0.05

In [53]:
result = solve_gurobi(
    start_hour = start_hour,
    end_hour = end_hour,
    instance = process_operations,
    contributions = contributions,
    E_max=250,
    alpha=0.9,
    beta=0.9,
    theta=0.005,
    G_max_solar=G_max_solar,
    G_max_grid= 1000,
    GPU_power_options=[100, 125, 150, 175, 200, 225, 250],
    electricity_price= electricity_price,
    sell_back_price=0.05
)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 421 rows, 7321 columns and 23070 nonzeros
Model fingerprint: 0x886b6e26
Variable types: 391 continuous, 6930 integer (6930 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+02]
  Objective range  [5e-02, 5e-01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 137 rows and 6755 columns
Presolve time: 0.01s
Presolved: 284 rows, 566 columns, 1665 nonzeros
Variable types: 176 continuous, 390 integer (360 binary)
Found heuristic solution: objective 361.8303583

Root relaxation: objective 3.143692e+02, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  314.36925    0   11  361.830

In [54]:

print(result)

    time      G_grid      W_grid      P_grid      R_grid  G_solar     W_solar  \
0     10    0.000000  548.037000    0.000000    0.000000  698.037  548.037000   
1     11    0.000000  673.739000    0.000000    0.000000  798.739  673.739000   
2     12    0.000000  344.470000    0.000000    0.000000  744.470  344.470000   
3     13    0.000000  374.736000    0.000000    0.000000  550.861  374.736000   
4     14    0.000000  123.002562    0.000000    0.000000  276.852  123.002562   
5     15    0.000000    0.000000    0.000000    0.000000   47.432    0.000000   
6     16    0.000000    0.000000    0.000000    0.000000    0.000    0.000000   
7     17   30.243066    0.000000   30.243066    0.000000    0.000    0.000000   
8     18  100.000000    0.000000  100.000000    0.000000    0.000    0.000000   
9     19  100.000000    0.000000  100.000000    0.000000    0.000    0.000000   
10    20  100.000000    0.000000  100.000000    0.000000    0.000    0.000000   
11    21  100.000000    0.00